In [ ]:
# Импортируем датасет

import pandas as pd

df = pd.read_csv("https://stepik.org/media/attachments/lesson/1028705/mulimodal_questions.csv")
df


In [ ]:
# Установим необходимую версию библиотеки

!pip install bitsandbytes

In [ ]:
# Скачаем zip-архив с картинками

!wget https://stepik.org/media/attachments/lesson/1028705/images.zip
!unzip images.zip

In [ ]:
from PIL import Image
import torch
from transformers import pipeline
from transformers import BitsAndBytesConfig

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, # подгружаем сразу оптимальную версию
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
# указываем тип задачи и модель
model_id = "llava-hf/llava-1.5-7b-hf"

pipe = pipeline("image-to-text", model=model_id, model_kwargs={"quantization_config": quantization_config})

In [ ]:
images = [f"images/im{i}.jpg" for i in range(0, 10)]

In [ ]:
# Получим предсказания для изображений

ans = []
for im_path, question in zip(images, df['question'].values):

    im = Image.open(im_path)

    prompt = f"USER:<image>\n{question}. answer with an int number\nASSISTANT:"
    outputs = pipe(im, prompt=prompt, generate_kwargs={"max_new_tokens": 200})
    outputs = int(outputs[0]['generated_text'].split('ASSISTANT: ')[1])
    ans.append(outputs)
    #break # уберите break, когда убедитесь, что код работает для одного изображения
print(ans)

In [ ]:
# запишем ответы в датафрейм

df['answer'] = ans
df.drop(columns=['image_name'], inplace=True)
df.to_csv('answer.csv', index=False)
df

In [ ]:
#---------------------------2-е решение-----------------------------------------------

import cv2
import torch
import pandas as pd

from PIL import Image
from transformers import AutoModel, AutoTokenizer


# Загружаем модель
model = AutoModel.from_pretrained('openbmb/MiniCPM-V-2', trust_remote_code=True, torch_dtype=torch.bfloat16)
model = model.to(device='cuda', dtype=torch.float16)

tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-V-2', trust_remote_code=True)
model = model.eval()

# Отвечаем на вопросы
df_answer = pd.read_csv("https://stepik.org/media/attachments/lesson/1028705/mulimodal_questions.csv")

for i in range(len(df_answer)):
    img = cv2.imread(df_answer.iloc[i, -1])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    im_pil = Image.fromarray(img)

    # Каждый вопрос ожидает только количество, поэтому можно использовать модель английского pre train. Для формата вывода добавляем свой промт к вопросу
    question = df_answer.iloc[i, 0] + " Get answer only number! Only one word!"
    msgs = [{'role': 'user', 'content': question}]

    res, context, _ = model.chat(
        image=im_pil,
        msgs=msgs,
        context=None,
        tokenizer=tokenizer,
        sampling=True,
        temperature=1
    )
    
    df_answer.iloc[i, 1] = res

df_answer.drop(columns="image_name").to_csv("solution_5.1.10.csv", index=False)